## <font color = 'red'>Ignore this, I'm restarting...T-T <font>

In [ ]:
import pandas as pd

def load_recipes(file_path):
    """Loads a recipe dataset from CSV or JSON."""
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith('.json'):
        return pd.read_json(file_path)
    else:
        raise ValueError("Unsupported file format. Use CSV or JSON.")

def filter_recipes(recipes, available_ingredients, available_tools, max_time):
    """Filters recipes based on ingredients, kitchen tools, and time constraints."""
    filtered = []
    
    for _, recipe in recipes.iterrows():
        required_ingredients = set(recipe['ingredients'].split(', '))
        required_tools = set(recipe['tools'].split(', '))
        prep_time = recipe['time']
        
        # Check ingredient availability
        if not required_ingredients.issubset(set(available_ingredients)):
            continue
        
        # Check kitchen tool availability
        if not required_tools.issubset(set(available_tools)):
            continue
        
        # Check time constraint
        if prep_time > max_time:
            continue
        
        filtered.append(recipe)
    
    return pd.DataFrame(filtered)

# Example Usage:
# recipes = load_recipes("recipes.csv")
# available_ingredients = ["eggs", "flour", "milk", "butter"]
# available_tools = ["oven", "whisk"]
# max_time = 30
# filtered_recipes = filter_recipes(recipes, available_ingredients, available_tools, max_time)
# print(filtered_recipes)




In [8]:
import os
print(os.listdir())  # Check available files in the working directory


['PP_recipes.csv', 'nikita_playground.ipynb', 'test.json', 'RAW_recipes.csv', 'train.json']


In [9]:
import pandas as pd

# Load datasets from local files
df1 = pd.read_csv("PP_recipes.csv")
df2 = pd.read_csv("RAW_recipes.csv")
df3 = pd.read_json("test.json")
df4 = pd.read_json("train.json")

# Store datasets in a dictionary for easier access
datasets = {
    "PP_recipes": df1,
    "RAW_recipes": df2,
    "Test_Recipes": df3,
    "Train_Recipes": df4
}

# Print dataset column names to understand their structure
for name, df in datasets.items():
    print(f"{name} Columns:", df.columns)

PP_recipes Columns: Index(['id', 'i', 'name_tokens', 'ingredient_tokens', 'steps_tokens',
       'techniques', 'calorie_level', 'ingredient_ids'],
      dtype='object')
RAW_recipes Columns: Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')
Test_Recipes Columns: Index(['id', 'ingredients'], dtype='object')
Train_Recipes Columns: Index(['id', 'cuisine', 'ingredients'], dtype='object')


In [10]:
# Drop rows with missing 'name' (since it's crucial)
datasets["RAW_recipes"].dropna(subset=["name"], inplace=True)

# Fill missing descriptions with placeholder text
datasets["RAW_recipes"]["description"].fillna("No description available", inplace=True)

# Verify missing values are handled
print("\n✅ After handling missing values:")
print(datasets["RAW_recipes"].isnull().sum())



✅ After handling missing values:
name              0
id                0
minutes           0
contributor_id    0
submitted         0
tags              0
nutrition         0
n_steps           0
steps             0
description       0
ingredients       0
n_ingredients     0
dtype: int64


/var/folders/v7/ql09t2fn277_jnyd8yjb2v880000gq/T/ipykernel_89601/1016164666.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datasets["RAW_recipes"]["description"].fillna("No description available", inplace=True)


In [11]:
import pandas as pd
import numpy as np

# Function to clean and preprocess the ingredients column
def clean_ingredients(ingredients):
    """Convert ingredients to lowercase, remove special characters, and format as a set."""
    if isinstance(ingredients, list):
        return set([ingredient.lower().strip() for ingredient in ingredients])
    elif isinstance(ingredients, str):
        return set([ingredient.lower().strip() for ingredient in ingredients.split(',')])
    else:
        return set()

# Apply cleaning to each dataset
for name, df in datasets.items():
    if "ingredients" in df.columns:
        df["ingredients"] = df["ingredients"].apply(clean_ingredients)

# Handle missing values
for name, df in datasets.items():
    print(f"🔍 Checking missing values in {name}:")
    print(df.isnull().sum())

    # Fill or drop missing values if necessary
    df.fillna("", inplace=True)  # Replace NaN with empty strings for text fields

# Check if datasets are now clean
for name, df in datasets.items():
    print(f"✅ Cleaned {name}: {df.head()}")


🔍 Checking missing values in PP_recipes:
id                   0
i                    0
name_tokens          0
ingredient_tokens    0
steps_tokens         0
techniques           0
calorie_level        0
ingredient_ids       0
dtype: int64
🔍 Checking missing values in RAW_recipes:
name              0
id                0
minutes           0
contributor_id    0
submitted         0
tags              0
nutrition         0
n_steps           0
steps             0
description       0
ingredients       0
n_ingredients     0
dtype: int64
🔍 Checking missing values in Test_Recipes:
id             0
ingredients    0
dtype: int64
🔍 Checking missing values in Train_Recipes:
id             0
cuisine        0
ingredients    0
dtype: int64
✅ Cleaned PP_recipes:        id       i                                        name_tokens  \
0  424415      23  [40480, 37229, 2911, 1019, 249, 6878, 6878, 28...   
1  146223   96900       [40480, 18376, 7056, 246, 1531, 2032, 40481]   
2  312329  120056     [40480, 2

In [18]:
import random
import pandas as pd

def standardize_datasets(datasets):
    """Ensure all datasets have a 'minutes' column for consistency."""
    
    for name, df in datasets.items():
        # Identify possible time-related columns
        possible_time_cols = ["minutes", "time", "cook_time", "prep_time"]

        # Find an existing time column
        time_col = next((col for col in possible_time_cols if col in df.columns), None)

        if time_col:
            # Rename the found column to 'minutes' for consistency
            df.rename(columns={time_col: "minutes"}, inplace=True)
        else:
            # If no time-related column exists, create 'minutes' with default 0
            df["minutes"] = 0

        # Print confirmation
        print(f"✅ Standardized dataset: {name} (Time column: 'minutes')")

    return datasets

# Run this before filtering or generating recipes
datasets = standardize_datasets(datasets)

for name, df in datasets.items():
    print(f"\n🔍 Checking dataset: {name}")
    print(df.head())  # Show sample rows
    if "minutes" not in df.columns:
        print(f"❌ ERROR: 'minutes' column is missing from {name}!")

def enforce_minutes_column(datasets):
    """Ensure all datasets contain a 'minutes' column as an integer."""
    for name, df in datasets.items():
        if "minutes" not in df.columns:
            print(f"⚠️ WARNING: 'minutes' missing in {name}. Adding default values.")
            df["minutes"] = 0  # Default value
        else:
            # Convert to integer type if it exists
            df["minutes"] = pd.to_numeric(df["minutes"], errors="coerce").fillna(0).astype(int)

    return datasets

# Run before filtering
datasets = enforce_minutes_column(datasets)

def ensure_minutes_column(datasets):
    """Forces every dataset to have a numeric 'minutes' column to prevent errors."""
    for name, df in datasets.items():
        if "minutes" not in df.columns:
            print(f"⚠️ Fixing: Adding 'minutes' column to {name}.")
            df["minutes"] = 0  # Default to 0 minutes if missing
        else:
            # Convert to integer to avoid issues
            df["minutes"] = pd.to_numeric(df["minutes"], errors="coerce").fillna(0).astype(int)

        print(f"✅ Verified: {name} has 'minutes' as numeric.")

    return datasets

# Apply before filtering
datasets = ensure_minutes_column(datasets)

def get_best_dataset(datasets, available_ingredients, available_tools, max_time):
    """Automatically selects the dataset with the most matching recipes."""
    dataset_scores = {}

    for name, dataset in datasets.items():
        if "ingredients" in dataset.columns:
            # Count recipes that match the ingredient and time constraints
            filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)
            dataset_scores[name] = len(filtered)

    if not dataset_scores or max(dataset_scores.values()) == 0:
        print("❌ No suitable dataset found.")
        return None

    # Choose the dataset with the most valid recipes
    best_dataset = max(dataset_scores, key=dataset_scores.get)

    print(f"\n✅ Using Best Dataset: {best_dataset} ({dataset_scores[best_dataset]} matching recipes)")
    return datasets[best_dataset]


# Debug: Check all dataset columns before filtering
for name, df in datasets.items():
    print(f"\n🔍 Dataset: {name} - Columns: {df.columns.tolist()}")
    

# def filter_recipes(recipes, available_ingredients, available_tools, max_time):
#     """Filters recipes based on available ingredients, kitchen tools, and time constraints."""
#     if recipes is None or recipes.empty:
#         print("❌ No recipes available to filter.")
#         return pd.DataFrame()

#     # Find the correct time-related column dynamically
#     possible_time_cols = ["minutes", "time", "cook_time"]
#     time_col = next((col for col in possible_time_cols if col in recipes.columns), None)

#     if time_col is None:
#         print("⚠️ No time-related column found. Skipping time filtering.")

#     filtered = []

#     for _, recipe in recipes.iterrows():
#         required_ingredients = recipe["ingredients"]

#         # Ensure 'tools' column exists
#         required_tools = set(recipe.get("tools", "").split(", ")) if "tools" in recipes.columns else set()

#         # Get preparation time if available
#         prep_time = int(recipe.get(time_col, 0)) if time_col else 0

#         # Check conditions
#         if not required_ingredients.issubset(set(available_ingredients)):
#             continue
#         if "tools" in recipes.columns and not required_tools.issubset(set(available_tools)):
#             continue
#         if time_col and prep_time > max_time:
#             continue

#         filtered.append(recipe)
def filter_recipes(recipes, available_ingredients, available_tools, max_time):
    """Filters recipes based on available ingredients, kitchen tools, and time constraints."""
    if recipes is None or recipes.empty:
        print("❌ No recipes available to filter.")
        return pd.DataFrame()

    # 🔹 Ensure 'minutes' still exists before filtering
    if "minutes" not in recipes.columns:
        print("⚠️ WARNING: 'minutes' column is missing! Re-adding it with default value 0.")
        recipes["minutes"] = 0  # Default value

    # Now continue filtering as usual
    filtered = []

    for _, recipe in recipes.iterrows():
        required_ingredients = recipe["ingredients"]
        required_tools = set(recipe.get("tools", "").split(", ")) if "tools" in recipes.columns else set()
        prep_time = int(recipe["minutes"])  # Now guaranteed to exist

        # Apply filters
        if not required_ingredients.issubset(set(available_ingredients)):
            continue
        if "tools" in recipes.columns and not required_tools.issubset(set(available_tools)):
            continue
        if prep_time > max_time:
            continue

        filtered.append(recipe)

    return pd.DataFrame(filtered).sort_values(by="minutes", ascending=True)


    # **Final Fix: Only sort if time_col exists**
    return pd.DataFrame(filtered).sort_values(by=time_col, ascending=True) if time_col else pd.DataFrame(filtered)

def generate_recipe(available_ingredients, available_tools, max_time, datasets):
    """Generates a new recipe based on available ingredients and constraints."""

    # Step 1: Automatically pick the best dataset
    dataset = get_best_dataset(datasets, available_ingredients, available_tools, max_time)

    if dataset is None:
        return "❌ No valid dataset found."

    # Step 2: Filter recipes
    filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)

    if filtered.empty:
        return "❌ No valid recipes found."

    # Step 3: Select a random recipe
    selected_recipe = filtered.sample(1).iloc[0]

    # Step 4: Randomly swap or add an ingredient
    ingredient_list = list(selected_recipe["ingredients"])
    if len(ingredient_list) > 2:
        ingredient_list[random.randint(0, len(ingredient_list) - 1)] = random.choice(list(available_ingredients))

    # Step 5: Format output (Fixing time_col lookup)
    time_col = next((col for col in ["minutes", "time", "cook_time"] if col in selected_recipe.index), None)

    return f"""
    🍽️ **Generated Recipe:** {selected_recipe['name']} (Modified)
    🕒 **Time:** {selected_recipe.get(time_col, 'Unknown')} minutes
    🛠️ **Tools:** {', '.join(available_tools)}
    📝 **Ingredients:** {', '.join(ingredient_list)}
    📜 **Steps:** {selected_recipe.get('steps', 'No steps available')}
    """

# Example Usage
available_ingredients = {"eggs", "flour", "milk", "butter", "chocolate"}
available_tools = {"oven", "whisk"}
max_time = 30

print(generate_recipe(available_ingredients, available_tools, max_time, datasets))

✅ Standardized dataset: PP_recipes (Time column: 'minutes')
✅ Standardized dataset: RAW_recipes (Time column: 'minutes')
✅ Standardized dataset: Test_Recipes (Time column: 'minutes')
✅ Standardized dataset: Train_Recipes (Time column: 'minutes')

🔍 Checking dataset: PP_recipes
       id       i                                        name_tokens  \
0  424415      23  [40480, 37229, 2911, 1019, 249, 6878, 6878, 28...   
1  146223   96900       [40480, 18376, 7056, 246, 1531, 2032, 40481]   
2  312329  120056     [40480, 21044, 16954, 8294, 556, 10837, 40481]   
3   74301  168258                       [40480, 10025, 31156, 40481]   
4   76272  109030  [40480, 17841, 252, 782, 2373, 1641, 2373, 252...   

                                   ingredient_tokens  \
0  [[2911, 1019, 249, 6878], [1353], [6953], [153...   
1  [[17918], [25916], [2507, 6444], [8467, 1179],...   
2  [[5867, 24176], [1353], [6953], [1301, 11332],...   
3  [[1270, 1645, 28447], [21601], [27952, 29471, ...   
4  [[1430

KeyError: 'minutes'

In [ ]:
# DATASET: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
# !pip install kagglehub
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path1 = "PP_recipes.csv"

# Load the latest version
df1 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "shuyangli94/food-com-recipes-and-user-interactions",
  file_path1
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df1.head())

In [ ]:
# DATASET: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
# !pip install kagglehub
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path2 = "RAW_recipes.csv"

# Load the latest version
df2 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "shuyangli94/food-com-recipes-and-user-interactions",
  file_path2
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df2.head())

In [ ]:
# DATASET: https://www.kaggle.com/datasets/kaggle/recipe-ingredients-dataset
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path3 = "test.json"

# Load the latest version
df3 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "kaggle/recipe-ingredients-dataset",
  file_path3,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df3.head())

In [ ]:
# DATASET: https://www.kaggle.com/datasets/kaggle/recipe-ingredients-dataset
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path4 = "train.json"

# Load the latest version
df4 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "kaggle/recipe-ingredients-dataset",
  file_path4,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df4.head())

In [ ]:
import random
import pandas as pd

def get_best_dataset(datasets, available_ingredients, available_tools, max_time):
    """Automatically selects the dataset with the most matching recipes."""
    dataset_scores = {}

    for name, dataset in datasets.items():
        if "ingredients" in dataset.columns:
            # Count recipes that match the ingredient and time constraints
            filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)
            dataset_scores[name] = len(filtered)

    if not dataset_scores or max(dataset_scores.values()) == 0:
        print("❌ No suitable dataset found.")
        return None

    # Choose the dataset with the most valid recipes
    best_dataset = max(dataset_scores, key=dataset_scores.get)

    print(f"\n✅ Using Best Dataset: {best_dataset} ({dataset_scores[best_dataset]} matching recipes)")
    return datasets[best_dataset]


In [ ]:
def filter_recipes(recipes, available_ingredients, available_tools, max_time):
    """Filters recipes based on available ingredients, kitchen tools, and time constraints."""
    if recipes is None or recipes.empty:
        print("❌ No recipes available to filter.")
        return pd.DataFrame()

    # Find the correct time-related column dynamically
    time_col = next((col for col in ["minutes", "time", "cook_time"] if col in recipes.columns), None)

    if time_col is None:
        print("⚠️ No time-related column found. Skipping time filtering.")

    filtered = []

    for _, recipe in recipes.iterrows():
        required_ingredients = recipe["ingredients"]

        # Ensure 'tools' column exists
        required_tools = set(recipe.get("tools", "").split(", ")) if "tools" in recipes.columns else set()

        # Get preparation time if available
        prep_time = int(recipe.get(time_col, 0)) if time_col else 0

        # Check conditions
        if not required_ingredients.issubset(set(available_ingredients)):
            continue
        if "tools" in recipes.columns and not required_tools.issubset(set(available_tools)):
            continue
        if time_col and prep_time > max_time:
            continue

        filtered.append(recipe)

    return pd.DataFrame(filtered).sort_values(by=time_col, ascending=True) if time_col else pd.DataFrame(filtered)

In [ ]:
def generate_recipe(available_ingredients, available_tools, max_time, datasets):
    """Generates a new recipe based on available ingredients and constraints."""

    # Step 1: Automatically pick the best dataset
    dataset = get_best_dataset(datasets, available_ingredients, available_tools, max_time)

    if dataset is None:
        return "❌ No valid dataset found."

    # Step 2: Filter recipes
    filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)

    if filtered.empty:
        return "❌ No valid recipes found."

    # Step 3: Select a random recipe
    selected_recipe = filtered.sample(1).iloc[0]

    # Step 4: Randomly swap or add an ingredient
    ingredient_list = list(selected_recipe["ingredients"])
    if len(ingredient_list) > 2:
        ingredient_list[random.randint(0, len(ingredient_list) - 1)] = random.choice(list(available_ingredients))

    # Step 5: Format output
    return f"""
    🍽️ **Generated Recipe:** {selected_recipe['name']} (Modified)
    🕒 **Time:** {selected_recipe.get(time_col, 'Unknown')} minutes
    🛠️ **Tools:** {', '.join(available_tools)}
    📝 **Ingredients:** {', '.join(ingredient_list)}
    📜 **Steps:** {selected_recipe.get('steps', 'No steps available')}
    """

# Example Usage
available_ingredients = {"eggs", "flour", "milk", "butter", "chocolate"}
available_tools = {"oven", "whisk"}
max_time = 30

print(generate_recipe(available_ingredients, available_tools, max_time, datasets))

In [ ]:
def filter_recipes(recipes, available_ingredients, available_tools, max_time):
    """Filters recipes based on available ingredients, kitchen tools, and time constraints."""
    if recipes is None or recipes.empty:
        print("❌ No recipes available to filter.")
        return pd.DataFrame()

    time_col = "minutes" if "minutes" in recipes.columns else None

    if time_col is None:
        print("❌ No time-related column found. Skipping time filtering.")

    filtered = []

    for _, recipe in recipes.iterrows():
        required_ingredients = recipe["ingredients"]

        # Ensure 'tools' column exists
        required_tools = set(recipe["tools"].split(", ")) if "tools" in recipes.columns else set()

        # Get preparation time
        prep_time = int(recipe[time_col]) if time_col else 0

        # Check conditions
        if not required_ingredients.issubset(set(available_ingredients)):
            continue
        if "tools" in recipes.columns and not required_tools.issubset(set(available_tools)):
            continue
        if time_col and prep_time > max_time:
            continue

        filtered.append(recipe)

    return pd.DataFrame(filtered).sort_values(by=time_col, ascending=True) if time_col else pd.DataFrame(filtered)

# Example usage
available_ingredients = {"eggs", "flour", "milk", "butter"}
available_tools = {"oven", "whisk"}
max_time = 30

filtered_recipes = filter_recipes(datasets["RAW_recipes"], available_ingredients, available_tools, max_time)
print("\n🔹 Filtered Recipes:\n", filtered_recipes.head())


In [ ]:
# Rule-Based Recipe Generation
import random

def generate_recipe(available_ingredients, available_tools, max_time, dataset):
    """Generates a new recipe based on available ingredients and constraints."""
    filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)
    
    if filtered.empty:
        return "❌ No valid recipes found."

    # Pick a random recipe from filtered results
    selected_recipe = filtered.sample(1).iloc[0]

    # Format the output
    return f"""
    🍽️ Recipe: {selected_recipe['name']}
    🕒 Time: {selected_recipe['minutes']} minutes
    🛠️ Tools: {', '.join(available_tools)}
    📝 Ingredients: {', '.join(selected_recipe['ingredients'])}
    📜 Steps: {selected_recipe['steps'] if 'steps' in selected_recipe else 'No steps available'}
    """

# Example Usage
print(generate_recipe({"eggs", "flour", "milk", "butter"}, {"oven", "whisk"}, 30, datasets["RAW_recipes"]))


In [ ]:
#AI-Based Recipe Generation (Python + NLP)
from transformers import pipeline

# Load pre-trained NLP model (T5 is good for text generation)
generator = pipeline("text-generation", model="t5-small")

def generate_ai_recipe(available_ingredients, available_tools, max_time):
    """Generates a recipe using an AI model (GPT/T5)."""
    
    prompt = f"Generate a recipe using:\nIngredients: {', '.join(available_ingredients)}\nTools: {', '.join(available_tools)}\nMax Time: {max_time} minutes"
    
    result = generator(prompt, max_length=150, num_return_sequences=1)
    
    return f"🍽️ AI-Generated Recipe:\n{result[0]['generated_text']}"

# Example Usage
print(generate_ai_recipe({"eggs", "flour", "milk", "butter"}, {"oven", "whisk"}, 30))


In [ ]:
# 🚀 Updated Rule-Based Recipe Generator (Fully Automated)

import random
import pandas as pd

def get_best_dataset(datasets, available_ingredients, available_tools, max_time):
    """Automatically selects the dataset with the most matching recipes."""
    dataset_scores = {}

    for name, dataset in datasets.items():
        if "ingredients" in dataset.columns:
            # Count recipes that match the ingredient and time constraints
            filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)
            dataset_scores[name] = len(filtered)

    # Choose the dataset with the most valid recipes
    best_dataset = max(dataset_scores, key=dataset_scores.get)

    print(f"\n✅ Using Best Dataset: {best_dataset} ({dataset_scores[best_dataset]} matching recipes)")
    return datasets[best_dataset]

def generate_recipe(available_ingredients, available_tools, max_time, datasets):
    """Generates a new recipe based on available ingredients and constraints."""

    # Step 1: Automatically pick the best dataset
    dataset = get_best_dataset(datasets, available_ingredients, available_tools, max_time)

    # Step 2: Filter recipes
    filtered = filter_recipes(dataset, available_ingredients, available_tools, max_time)

    if filtered.empty:
        return "❌ No valid recipes found."

    # Step 3: Select a random recipe
    selected_recipe = filtered.sample(1).iloc[0]

    # Step 4: Randomly swap or add an ingredient
    ingredient_list = list(selected_recipe["ingredients"])
    if len(ingredient_list) > 2:
        ingredient_list[random.randint(0, len(ingredient_list) - 1)] = random.choice(list(available_ingredients))

    # Step 5: Format output
    return f"""
    🍽️ **Generated Recipe:** {selected_recipe['name']} (Modified)
    🕒 **Time:** {selected_recipe['minutes']} minutes
    🛠️ **Tools:** {', '.join(available_tools)}
    📝 **Ingredients:** {', '.join(ingredient_list)}
    📜 **Steps:** {selected_recipe['steps'] if 'steps' in selected_recipe else 'No steps available'}
    """

# Example Usage
available_ingredients = {"eggs", "flour", "milk", "butter", "chocolate"}
available_tools = {"oven", "whisk"}
max_time = 30

print(generate_recipe(available_ingredients, available_tools, max_time, datasets))


In [ ]:
import kagglehub
import pandas as pd
from kagglehub import KaggleDatasetAdapter

# Load datasets from Kaggle
datasets = {
    "PP_recipes": df1,
    "RAW_recipes": df2,
    "Test_Recipes": df3,
    "Train_Recipes": df4
}

loaded_data = {}

for name, file_path in datasets.items():
    try:
        loaded_data[name] = kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            "shuyangli94/food-com-recipes-and-user-interactions" if "recipes" in name else "kaggle/recipe-ingredients-dataset",
            file_path
        )
        print(f"✅ Loaded {name} successfully. First 5 rows:")
        print(loaded_data[name].head())
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")

# Function to load datasets from local files (fallback method)
def load_recipes(file_path):
    """Loads a recipe dataset from CSV or JSON."""
    try:
        if file_path.endswith('.csv'):
            return pd.read_csv(file_path)
        elif file_path.endswith('.json'):
            return pd.read_json(file_path)
        else:
            raise ValueError("Unsupported file format. Use CSV or JSON.")
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None

# Recipe filtering function
def filter_recipes(recipes, available_ingredients, available_tools, max_time):
    """Filters recipes based on available ingredients, kitchen tools, and time constraints."""
    if recipes is None or recipes.empty:
        print("❌ No recipes available to filter.")
        return pd.DataFrame()

    filtered = []

    for _, recipe in recipes.iterrows():
        # Handle missing values safely
        required_ingredients = set(str(recipe.get('ingredients', '')).lower().split(', '))
        required_tools = set(str(recipe.get('tools', '')).lower().split(', '))
        prep_time = int(recipe.get('time', 0))  # Ensure time is an integer

        # Check ingredient availability
        if not required_ingredients.issubset(set(available_ingredients)):
            continue

        # Check kitchen tool availability
        if not required_tools.issubset(set(available_tools)):
            continue

        # Check time constraint
        if prep_time > max_time:
            continue

        filtered.append(recipe)

    return pd.DataFrame(filtered).sort_values(by="time", ascending=True)

# Example Usage
available_ingredients = {"eggs", "flour", "milk", "butter"}
available_tools = {"oven", "whisk"}
max_time = 30

# Filter recipes using one of the loaded datasets (choose appropriately)
if "PP_recipes" in loaded_data:
    filtered_recipes = filter_recipes(loaded_data["PP_recipes"], available_ingredients, available_tools, max_time)
    print("Filtered Recipes:\n", filtered_recipes.head())

